- Cleaning Lexicons 
- Finding SPs

## Filepaths

In [1]:
repopath = '../repo'
datapath = f'{repopath}/data'
flp = f'{datapath}/merged_food_list.csv'
menup = f'{datapath}/menu.csv'
cvaw4p = f'{datapath}/cvaw4.csv'

In [2]:
parsedp = f'{datapath}/review_clean_parsed.csv'

In [3]:
# !pip install pandas

In [4]:
import pandas as pd
import numpy as np

In [5]:
parse = pd.read_csv(parsedp, converters={'word_seg': pd.eval, 'pos': pd.eval, 'dependency_parse':pd.eval})
                    # https://stackoverflow.com/questions/23111990/pandas-dataframe-stored-list-as-string-how-to-convert-back-to-list
menu = pd.read_csv(menup)
fl = pd.read_csv(flp)
sentilex = pd.read_csv(cvaw4p)

In [6]:
# extending foodlist: merging foodlist and beefnoddles menu
fl = fl[fl['source'] != 'eHowNet-cosmetics']
foods = pd.concat([menu['name'], fl['food_list']])
foods = foods.to_list() 
sentis = sentilex['Word'].tolist()
# food lexicon 

In [7]:
# defining a (temporary) opinion lexicon based on 
custom_opinions = {'鹹':3, 
                   '香':5,
                   '親切':5,
                   '好':5,
                   '差':1, '佳':5, '少':1, '夠':5, '多':4, '棒':5, '不錯':5}
sentis.extend(custom_opinions.keys())

In [8]:
parse

,Unnamed: 0,sentence,index,length,time,rating,word_seg,pos,dependency_parse
0,0,酸辣湯是酸甜口感,1,8,1年前,3,"[酸辣湯, 是, 酸甜, 口感]","[[0, root root], [1, 酸辣湯 Na], [2, 是 SHI], [3, ...","[[2 - 是 SHI, 1 - 酸辣湯 Na, amod], [0 - root root..."
1,1,牛肉湯麵的湯有點太鹹了,1,11,1年前,3,"[牛肉湯麵, 的, 湯, 有點, 太, 鹹, 了]","[[0, root root], [1, 牛肉湯麵 Na], [2, 的 DE], [3, ...","[[6 - 鹹 VH, 1 - 牛肉湯麵 Na, nn], [4 - 有點 Dfa, 2 -..."
2,2,整體吃起來感覺普普,1,9,1年前,3,"[整體, 吃起來, 感覺, 普普]","[[0, root root], [1, 整體 Na], [2, 吃起來 D], [3, 感...","[[2 - 吃起來 D, 1 - 整體 Na, nsubj], [4 - 普普 Nb, 2 ..."
3,3,喜歡吃帶筋的牛肉的話會是一個好選擇,2,17,2年前,4,"[喜歡, 吃, 帶, 筋, 的, 牛肉, 的話, 會, 是, 一, 個, 好, 選擇]","[[0, root root], [1, 喜歡 VK], [2, 吃 VC], [3, 帶 ...","[[9 - 是 SHI, 1 - 喜歡 VK, dep], [1 - 喜歡 VK, 2 - ..."
4,4,牛肉湯滿清的但口味比較偏鹹一些,2,15,2年前,4,"[牛肉湯, 滿清, 的, 但, 口味, 比較, 偏, 鹹, 一些]","[[0, root root], [1, 牛肉湯 Na], [2, 滿清 Nb], [3, ...","[[2 - 滿清 Nb, 1 - 牛肉湯 Na, nn], [0 - root root, ..."
...,...,...,...,...,...,...,...,...,...
96,96,味道蠻喜歡的,19,6,3年前,4,"[味道, 蠻, 喜歡, 的]","[[0, root root], [1, 味道 Na], [2, 蠻 Dfa], [3, 喜...","[[3 - 喜歡 VK, 1 - 味道 Na, nsubj], [3 - 喜歡 VK, 2 ..."
97,97,份量算少大食量的人可能吃不飽,19,14,3年前,4,"[份量, 算, 少, 大, 食量, 的, 人, 可能, 吃, 不, 飽]","[[0, root root], [1, 份量 Na], [2, 算 VG], [3, 少 ...","[[2 - 算 VG, 1 - 份量 Na, nsubj], [9 - 吃 VC, 2 - ..."
98,98,算不錯,20,3,1年前,3,"[算, 不錯]","[[0, root root], [1, 算 VG], [2, 不錯 VH]]","[[0 - root root, 1 - 算 VG, root], [1 - 算 VG, 2..."
99,99,但是不知是辣椒醬太鹹,20,10,1年前,3,"[但是, 不, 知, 是, 辣椒醬, 太, 鹹]","[[0, root root], [1, 但是 Cbb], [2, 不 D], [3, 知 ...","[[3 - 知 VK, 1 - 但是 Cbb, advmod], [3 - 知 VK, 2 ..."


## Detecting pairs

In [9]:

def detect(sentpos):
    foodidx, foodtok = [], []
    sentiidx, sentitok = [], []
    
    for x in sentpos:
        id, tokenpos = x
        token, pos = tokenpos.split()
        if token in foods:
            foodidx.append(id)
            foodtok.append(token)
        if token in sentis:
            sentiidx.append(id)
            sentitok.append(token)
        food = {'idx': foodidx, 'token':foodtok}
        senti = {'idx': sentiidx, 'token':sentitok}
    # res = True if (foodidx and sentiidx) else False
    return food, senti      

In [10]:
!pip install networkx

    every sentence is a graph G 
    抓所有的 aspects，
    好像要幫兩點之間的連線記 direction
    可以用 拿回relation關係 
    visualize時印 {token} -{dep relation}- {token} -{dep relation}-{token} 
    有辦法把 dependency

In [71]:
# 還是湯頭太鹹
# ['還是', '湯頭', '太', '鹹']
# aspect: {'idx': [2], 'token': ['湯頭']}
# opinion: {'idx': [4], 'token': ['鹹']}
# [2, 4]

def process_sp(graph, sp):
    dirpath = []
    for i in range(1, len(sp)):
        u, v = sp[i-1], sp[i]
        # ustring = f'{u} - {G.nodes[u]["word"]} {G.nodes[u]["pos"]}'
        if graph.has_edge(u, v): 
            deprel = G.get_edge_data(u, v)['dep_relation']
            dirpath.append((u, v, deprel))
        else: 
            deprel = G.get_edge_data(v, u)['dep_relation']
            dirpath.append((v, u, deprel))
    
    viewpath = []
    for i in range(1, len(sp)):
        edge = u, v = sp[i-1], sp[i]
        ustring = f'{u} - {G.nodes[u]["word"]} {G.nodes[u]["pos"]}'
        vstring = f'{v} - {G.nodes[v]["word"]} {G.nodes[v]["pos"]}'
        viewpath.append(f'{ustring} --> {vstring}')
    return dirpath, viewpath

In [83]:
%%capture cap --no-stderr

import networkx as nx
from collections import defaultdict
from copy import deepcopy
treeD = defaultdict(dict)
for rowid, r in parse.iterrows():
    
    sentence, ws, pos, depp = r['sentence'], r['word_seg'], r['pos'], r['dependency_parse']
    nodeset = []
    aspect, opinion = detect(pos)
   
    # graph-building 
    # if aspect['idx'] and opinion['idx']:
    
    
    print('=======')
    print(f'text: {sentence}')
    print(f'ws: {ws}')
    print('aspect(s):', aspect)
    print('opinion(s):', opinion)

    G = nx.DiGraph()
    for x in pos:
        id, tokenpos = x
        token, pos = tokenpos.split()
        node = (id, {'word':token, 'pos':pos})
        nodeset.append(node)
    G.add_nodes_from(nodeset)
    for dep in depp:
        u, v, deprel = dep 
        u, v = u.split(' - '), v.split(' - ')
        uid = u[0] = int(u[0])
        vid = v[0] = int(v[0])
        G.add_edge(uid, vid, dep_relation = deprel)
    # nx.draw(G)
    copyG = deepcopy(G)
    copyG = copyG.to_undirected()
    treeD[rowid]['sentence'] = sentence
    treeD[rowid]['word seg'] = ws
    treeD[rowid]['pos'] = pos
    treeD[rowid]['dependency parse'] = depp
    treeD[rowid]['graph'] = G
    
    for asp, asptok in zip(aspect['idx'], aspect['token']):
        for opn, opntok in zip(opinion['idx'], opinion['token']):
            # find SP
            sp = nx.shortest_path(copyG, asp, opn, method = 'dijkstra')
            # Supported options: ‘dijkstra’, ‘bellman-ford’. Other inputs produce a ValueError.
            directedpath, viewpath = process_sp(G, sp)
            #  (consider no tree direction)
            print(f'Path from {asptok}_{asp} to {opntok}_{opn}:')
            print(f'\t{directedpath},\n\t{viewpath}')

with open('../data/0702_absa_review_sp.txt', 'w') as f:
    f.write(str(cap))           

In [82]:
# G = nx.path_graph(3)
# attrs = {(0, 1): {"attr1": 20, "attr2": "nothing"}, (1, 2): {"attr2": 3}}
# nx.set_edge_attributes(G, attrs)
# G[0][1]["attr1"]
# 20
# G[0][1]["attr2"]
# 'nothing'
# G[1][2]["attr2"]
# 3

In [ ]:
!pip install matplotlib